In [ ]:
# Author: Arthur BARREAU
# Date: 2025-07-15
# R version: 4.3.3
# Description: This script takes a date range and subarea as input,and outputs
#              a text file containing the CDS API request parameters.

# ==============================================================================
# 0. PACKAGE & LIBRARY INSTALLATION ----
# ==============================================================================

system("conda install -y r-sf=1.0_20")

library(sf)
library(dplyr)
library(jsonlite)

# ==============================================================================
# 1. GLOBAL OPTIONS & CONFIG LOADING ----
# ==============================================================================
options(timeout = 3600)

config <- fromJSON("galaxy_inputs/galaxy_inputs.json")

# Extract and format year range
year_range <- as.numeric(unlist(strsplit(config$year, ",")))
year_list <- seq(year_range[1], year_range[2])
formatted_years <- sprintf("\"%s\"", year_list)
year_csv <- paste(formatted_years, collapse = ", ")

# Extract and format subareas
subarea_list <- unlist(strsplit(config$subarea, ","))

# ==============================================================================
# 2. DOWNLOAD AND LOAD ASD SHAPEFILES ----
# ==============================================================================
download_asd_shapefiles <- function(output_dir = "asd") {
  dir.create(output_dir, showWarnings = FALSE)

  base_url <- "https://raw.githubusercontent.com/ccamlr/data/refs/tags/v0.5.0/geographical_data/asd/"
  files <- c("asd-shapefile-EPSG4326.shp",
             "asd-shapefile-EPSG4326.shx",
             "asd-shapefile-EPSG4326.dbf",
             "asd-shapefile-EPSG4326.prj",
             "asd-shapefile-EPSG4326.cst")

  urls <- paste0(base_url, files)
  destinations <- file.path(output_dir, files)

  mapply(download.file, urls, destinations, MoreArgs = list(mode = "wb"))

  st_read(file.path(output_dir, "asd-shapefile-EPSG4326.shp"))
}

asd_data <- download_asd_shapefiles()

# ==============================================================================
# 3. FILTER SUBAREAS & DEFINE BBOX ----
# ==============================================================================
selected_areas <- filter(asd_data, GAR_Short_ %in% subarea_list)
bbox <- st_bbox(selected_areas)

# Ensure correct bounding box order
xmin <- min(bbox["xmin"], bbox["xmax"])
xmax <- max(bbox["xmin"], bbox["xmax"])
ymin <- min(bbox["ymin"], bbox["ymax"])
ymax <- max(bbox["ymin"], bbox["ymax"])

# ==============================================================================
# 4. GENERATE PYTHON SCRIPT FOR CDS API ----
# ==============================================================================
generate_cds_script <- function(output_file, years_csv, bbox_coords) {
  script_text <- paste(
    "import cdsapi\n\n",
    "c = cdsapi.Client()\n",
    "c.retrieve(\n",
    "    \"reanalysis-era5-single-levels\",\n",
    "    {\n",
    "        \"product_type\": [\"ensemble_mean\"],\n",
    "        \"variable\": [\"sea_surface_temperature\"],\n",
    "        \"year\": [", years_csv, "],\n",
    "        \"month\": [\"01\", \"02\", \"03\", \"04\", \"05\", \"06\", \"07\", \"08\", \"09\", \"10\", \"11\", \"12\"],\n",
    "        \"day\": [\"01\", \"02\", \"03\", \"04\", \"05\", \"06\", \"07\", \"08\", \"09\", \"10\", \"11\", \"12\", \"13\", \"14\", \"15\", \"16\", \"17\", \"18\", \"19\", \"20\", \"21\", \"22\", \"23\", \"24\", \"25\", \"26\", \"27\", \"28\", \"29\", \"30\", \"31\"],\n",
    "        \"time\": [\"00:00\", \"03:00\", \"06:00\", \"09:00\", \"12:00\", \"15:00\", \"18:00\", \"21:00\"],\n",
    "        \"data_format\": \"grib\",\n",
    "        \"download_format\": \"unarchived\",\n",
    "        \"area\": [", bbox_coords$ymax, ", ", bbox_coords$xmin, ", ", bbox_coords$ymin, ", ", bbox_coords$xmax, "]\n",
    "    },\n",
    "    'download.nc'\n",
    ")"
  )

  writeLines(script_text, output_file)
}

# ==============================================================================
# 5. EXPORT SCRIPT ----
# ==============================================================================
output_script_path <- "outputs/script.txt"
generate_cds_script(
  output_file = output_script_path,
  years_csv = year_csv,
  bbox_coords = list(xmin = xmin, xmax = xmax, ymin = ymin, ymax = ymax)
)
